# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     Juan Álvaro Morales Ramirez Valadez|   |
| **Fecha**     06/09/2025 |   |
| **Expediente**745903 |   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [64]:
import pandas as pd
import numpy as np

In [66]:
df = pd.read_csv("Advertising.csv")

### EDA

In [69]:
df.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  200 non-null    int64  
 1   TV          200 non-null    float64
 2   radio       200 non-null    float64
 3   newspaper   200 non-null    float64
 4   sales       200 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 7.9 KB


In [73]:
df.describe()

,Unnamed: 0,TV,radio,newspaper,sales
count,200.000000,200.000000,200.000000,200.000000,200.000000
mean,100.500000,147.042500,23.264000,30.554000,14.022500
std,57.879185,85.854236,14.846809,21.778621,5.217457
min,1.000000,0.700000,0.000000,0.300000,1.600000
25%,50.750000,74.375000,9.975000,12.750000,10.375000
50%,100.500000,149.750000,22.900000,25.750000,12.900000
75%,150.250000,218.825000,36.525000,45.100000,17.400000
max,200.000000,296.400000,49.600000,114.000000,27.000000


### Limpieza de df

In [76]:
df = df.dropna()

df = df.drop_duplicates()

df.describe()

,Unnamed: 0,TV,radio,newspaper,sales
count,200.000000,200.000000,200.000000,200.000000,200.000000
mean,100.500000,147.042500,23.264000,30.554000,14.022500
std,57.879185,85.854236,14.846809,21.778621,5.217457
min,1.000000,0.700000,0.000000,0.300000,1.600000
25%,50.750000,74.375000,9.975000,12.750000,10.375000
50%,100.500000,149.750000,22.900000,25.750000,12.900000
75%,150.250000,218.825000,36.525000,45.100000,17.400000
max,200.000000,296.400000,49.600000,114.000000,27.000000


### Escalar Variables

In [79]:
x = df[['TV', 'radio', 'newspaper']]
y = df['sales']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(x)

x_scaled = scaler.transform(x)

# Modelo 1.- Sin penalizacion

In [105]:
import statsmodels.api as sm

n = 200

unos = np.ones([n,1])

X = np.hstack([
    unos,
    x
])

modelo = sm.OLS(y, X) #fit es al reves, es decir, primero Y y luego X

results = modelo.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        12:52:52   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [84]:
# Resultados modelo sin penalizar
print("Coeficientes:", [0.0458, 0.1885, -0.0010])
print("Intercepto:", 2.9389)
print("R²:", 0.897)

Coeficientes: [0.0458, 0.1885, -0.001]
Intercepto: 2.9389
R²: 0.897


# Modelo 2 con penalizacion L2 (Ridge)

In [87]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

In [89]:
# Entrenar el modelo Ridge
ridge = Ridge(alpha=0.5)
ridge.fit(x_scaled, y)

# Predicciones y R²
y_pred_ridge = ridge.predict(x_scaled)
r2_ridge = r2_score(y, y_pred_ridge)

# Resultados Ridge
print("Ridge Coeficientes:", ridge.coef_)
print("Ridge Intercepto:", ridge.intercept_)
print("R² Ridge:", r2_ridge)

Ridge Coeficientes: [ 3.9097101   2.78446328 -0.01925884]
Ridge Intercepto: 14.0225
R² Ridge: 0.8972052353258027


# Modelo 3 con penalizacion L1 (Lasso)

In [92]:
from sklearn.linear_model import Lasso

In [94]:
# Entrenar el modelo Lasso
lasso = Lasso(alpha=0.5)
lasso.fit(x_scaled, y)

# Predicciones y R²
y_pred_lasso = lasso.predict(x_scaled)
r2_lasso = r2_score(y, y_pred_lasso)

# Resultados Lasso
print("Lasso Coeficientes:", lasso.coef_)
print("Lasso Intercepto:", lasso.intercept_)
print("R² Lasso:", r2_lasso)

Lasso Coeficientes: [3.44439218 2.31010826 0.        ]
Lasso Intercepto: 14.0225
R² Lasso: 0.8796935559537357


# Comparar Modelos

In [97]:
# Resultados modelo sin penalizar
print("Coeficientes:", [0.0458, 0.1885, -0.0010])
print("Intercepto:", 2.9389)
print("R²:", 0.897)

Coeficientes: [0.0458, 0.1885, -0.001]
Intercepto: 2.9389
R²: 0.897


In [99]:
# Resultados Ridge
print("Ridge Coeficientes:", ridge.coef_)
print("Ridge Intercepto:", ridge.intercept_)
print("R² Ridge:", r2_ridge)

Ridge Coeficientes: [ 3.9097101   2.78446328 -0.01925884]
Ridge Intercepto: 14.0225
R² Ridge: 0.8972052353258027


In [101]:
# Resultados Lasso
print("Lasso Coeficientes:", lasso.coef_)
print("Lasso Intercepto:", lasso.intercept_)
print("R² Lasso:", r2_lasso)

Lasso Coeficientes: [3.44439218 2.31010826 0.        ]
Lasso Intercepto: 14.0225
R² Lasso: 0.8796935559537357


# Conclusión

En este ejercicio estuve probando tres modelos de regresión: uno sin penalización, uno con Ridge (penalización L2) y otro con Lasso (penalización L1). Algo que noté es que en todos los modelos el valor de R² fue bastante alto, entre 0.87 y 0.89, lo cual me indica que todos hacen un buen trabajo explicando las ventas a partir de la publicidad, aunque hay algunas diferencias entre ellos.

También vi que en el modelo sin penalización, los p-values me ayudaron a ver qué variables son significativas. En este caso, tanto TV como radio tuvieron un p-value de 0, lo que indica que sí aportan al modelo, pero el p-value de newspaper fue 0.86, lo cual es muy alto y eso sugiere que tal vez esa variable no es tan útil para predecir ventas.

Con los modelos Ridge y Lasso tuve que elegir un valor para alpha, que es lo que controla la penalización. La verdad no estaba muy seguro de cuál usar, así que puse alpha = 0.5 porque me pareció un valor intermedio y equilibrado entre no penalizar mucho pero tampoco exagerar.

Lo que me pareció interesante es que cuando usé Lasso, el coeficiente de newspaper salió exactamente en cero, lo cual creo que confirma lo que ya se veía en el p-value del modelo sin penalización: que esa variable no es tan importante, y Lasso directamente la "elimina" del modelo.

Nota: No pude sacarle los p-values al lasso ni al ridge, intente como el lab 3 pero no me daba. (Si se puede pasar el ejercicio resuleto para estudiar estaria chido)

Para finalizar tampoco sue si es mejor hacer el ejercicio sepranado los datos con train y test, asi que mejor lo deje normal.

Tambien note que en el modelo base que nos pedian para hacer las regresiones se incluia una variable de error, y la verdad no supe como ponerla en mis modelos